In [31]:
import glob 
import csv
import pandas as pd
import numpy as np
from scipy import stats 

import plotly.graph_objects as go
import plotly.express as px

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

In [32]:
merged_csvs = glob.glob('../raw_data_files_clean/merged_*')
merged_csvs.sort()

merged_csvs

['../raw_data_files_clean/merged_act_school.csv',
 '../raw_data_files_clean/merged_assessment.csv',
 '../raw_data_files_clean/merged_census_school.csv',
 '../raw_data_files_clean/merged_chronic_absenteeism_school.csv',
 '../raw_data_files_clean/merged_profile.csv',
 '../raw_data_files_clean/merged_tvaas_composite_school.csv',
 '../raw_data_files_clean/merged_tvaas_composite_subject_school.csv']

In [33]:
def pull_files_and_create_df(raw_file_names): 
    df_list=[]
    for i in raw_file_names:
        name = i.split('/')[2].split('.')[0]
        globals()[f'df_{name}'] = pd.read_csv(i, low_memory=False)
        d = df_list.append(f'df_{name}')
    df_list.sort(reverse=True)
    return df_list

In [34]:
pull_files_and_create_df(merged_csvs)

['df_merged_tvaas_composite_subject_school',
 'df_merged_tvaas_composite_school',
 'df_merged_profile',
 'df_merged_chronic_absenteeism_school',
 'df_merged_census_school',
 'df_merged_assessment',
 'df_merged_act_school']

In [73]:
df_merged_act_school

,district_number,district_name,school_number,school_name,subgroup,valid_tests,participation_rate,average_english_score,average_math_score,average_reading_score,average_science_score,average_composite_score,percent_scoring_21_or_higher,percent_scoring_below_19,file_source,file_year,updated_district_number,school_type,status
0,10,Anderson County,2,Anderson County High School,All Students,255,96,18.6,18.4,19.5,19.6,19.1,36.5,50.2,df_act_school_2021,2021,10,Public,A
1,10,Anderson County,2,Anderson County High School,Black/Hispanic/Native American,6,*,*,*,*,*,*,*,*,df_act_school_2021,2021,10,Public,A
2,10,Anderson County,2,Anderson County High School,Economically Disadvantaged,80,95,17,17,17.8,18.1,17.6,27.5,62.5,df_act_school_2021,2021,10,Public,A
3,10,Anderson County,2,Anderson County High School,English Learners,1,*,*,*,*,*,*,*,*,df_act_school_2021,2021,10,Public,A
4,10,Anderson County,2,Anderson County High School,Students with Disabilities,30,97,12.8,15.1,14.2,15,14.4,10,86.7,df_act_school_2021,2021,10,Public,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8886,985,NaN,8140,Hillcrest High School,All Students,82,85.0,14.5,15.2,15.4,16,15.4,**,89,df_act_school_2017,2017,985,Public Charter,A
8887,985,NaN,8140,Hillcrest High School,Black/Hispanic/Native American,80,85.0,14.4,15.2,15.3,16,15.3,**,90,df_act_school_2017,2017,985,Public Charter,A
8888,985,NaN,8140,Hillcrest High School,Economically Disadvantaged,53,82.0,13.7,14.8,15.5,15.6,15,**,94.3,df_act_school_2017,2017,985,Public Charter,A
8889,985,NaN,8140,Hillcrest High School,English Language Learners with T1/T2,1,100.0,*,*,*,*,*,*,*,df_act_school_2017,2017,985,Public Charter,A


## ACT

In [128]:
df_mnps_act = df_merged_act_school.loc[df_merged_act_school['updated_district_number']==190].copy()
df_mnps_act['school_type'] = df_mnps_act['school_type'].replace('Public Virtual School','Public')

mnps_missing_act_tests = df_mnps_act.loc[df_mnps_act['percent_scoring_below_19']=='*'].groupby(by=['file_year','subgroup']).sum('valid_tests').reset_index().pivot(index='file_year',columns='subgroup',values='valid_tests')
mnps_all_act_tests = df_mnps_act.groupby(by=['file_year','subgroup']).sum('valid_tests').reset_index().pivot(index='file_year',columns='subgroup',values='valid_tests')
mnps_missing_pct_act = pd.concat([mnps_all_act_tests, mnps_missing_act_tests],axis=1)
mnps_missing_pct_act.columns = ['all','poc','econ_disadv','el_with_t1_t2','el','swd','all_missing','poc_missing','econ_disadv_missing','el_with_t1_t2_missing','el_missing','swd_missing']
mnps_missing_pct_act['acc_pct'] = ((mnps_missing_pct_act['all_missing'] / mnps_missing_pct_act['all']) * 100)

# df_mnps_act = df_mnps_act.loc[df_mnps_act['percent_scoring_below_19']!='*']
df_mnps_act = df_mnps_act.replace(['*'],[0])

# df_mnps_act['missing_19'] = pd.np.where(df_mnps_act.percent_scoring_below_19.str.contains('*'), 'missing')
for v in ['valid_tests','participation_rate','average_english_score','average_math_score','average_reading_score','average_science_score','average_composite_score','percent_scoring_21_or_higher','percent_scoring_below_19']:
    df_mnps_act[v] = pd.to_numeric(df_mnps_act[v],errors='coerce') 
    
df_mnps_act['cnt_scoring_21_or_higher']  = ((df_mnps_act['percent_scoring_21_or_higher'] / 100) * df_mnps_act['valid_tests']).round(0)
df_mnps_act['cnt_scoring_19_or_higher']  = ((1 - (df_mnps_act['percent_scoring_below_19'] / 100)) * df_mnps_act['valid_tests']).round(0)
df_mnps_act['composite_score_total']  = (df_mnps_act['average_composite_score']) * (df_mnps_act['valid_tests'])


df_mnps_act_19_21 = df_mnps_act[['school_number','school_name','subgroup','valid_tests','file_year','cnt_scoring_21_or_higher','cnt_scoring_19_or_higher','composite_score_total','school_type']]
df_mnps_act_19_21 = df_mnps_act_19_21.pivot_table(index=['file_year'],columns=['school_type','subgroup'],values=['cnt_scoring_21_or_higher','cnt_scoring_19_or_higher','composite_score_total','valid_tests'],aggfunc=np.sum).reset_index()
df_mnps_act_19_21.columns = ['file_year',
                             'all_public_19', 'poc_public_19', 'econ_disadv_public_19','el_t1_t2_public_19','el_public_19','swd_public_19',
                             'all_charter_19', 'poc_charter_19', 'econ_disadv_charter_19','el_t1_t2_charter_19','el_charter_19','swd_charter_19',
                             
                             'all_public_21', 'poc_public_21', 'econ_disadv_public_21','el_t1_t2_public_21','el_public_21','swd_public_21',
                             'all_charter_21', 'poc_charter_21', 'econ_disadv_charter_21','el_t1_t2_charter_21','el_charter_21','swd_charter_21',
                             
                             'all_public_composite', 'poc_public_composite', 'econ_disadv_public_composite','el_t1_t2_public_composite','el_public_composite','swd_public_composite',
                             'all_charter_composite', 'poc_charter_composite', 'econ_disadv_charter_composite','el_t1_t2_charter_composite','el_charter_composite','swd_charter_composite',
                             
                             'all_public', 'poc_public', 'econ_disadv_public','el_t1_t2_public','el_public','swd_public',
                             'all_charter', 'poc_charter', 'econ_disadv_charter','el_t1_t2_charter','el_charter','swd_charter']

for i in ['all', 'poc', 'econ_disadv','el_t1_t2','swd']: 
    df_mnps_act_19_21[i+'_public_19_pct'] = ((df_mnps_act_19_21[i+'_public_19']/df_mnps_act_19_21[i+'_public'])*100).round(0)
    df_mnps_act_19_21[i+'_public_21_pct'] = ((df_mnps_act_19_21[i+'_public_21']/df_mnps_act_19_21[i+'_public'])*100).round(0)
    df_mnps_act_19_21[i+'_charter_19_pct'] = ((df_mnps_act_19_21[i+'_charter_19']/df_mnps_act_19_21[i+'_charter'])*100).round(0)
    df_mnps_act_19_21[i+'_charter_21_pct'] = ((df_mnps_act_19_21[i+'_charter_21']/df_mnps_act_19_21[i+'_charter'])*100).round(0)
    
    df_mnps_act_19_21[i+'_public_composite_avg'] = ((df_mnps_act_19_21[i+'_public_composite']/df_mnps_act_19_21[i+'_public'])).round(0)
    df_mnps_act_19_21[i+'_charter_composite_avg'] = ((df_mnps_act_19_21[i+'_charter_composite']/df_mnps_act_19_21[i+'_charter'])).round(0)
                                  
# df_mnps_act_avg = df_mnps_act_avg.groupby(by=['file_year','school_type','subgroup']).sum(['average_composite_score']).reset_index().pivot(index='file_year',columns=['school_type','subgroup']).reset_index()

# df_mnps_act_avg.columns=['file_year','all_public','poc_public','econ_disadv_public','el_t1_t2_public','swd_public','all_charter','poc_charter','econ_disadv_charter','el_t1_t2_charter','swd_charter','el_public','el_charter']


# df_mnps_act[['file_year','subgroup','school_type','participation_rate']].groupby(by=['file_year','subgroup','school_type']).mean('participation_rate').reset_index().head()

# df_mnps_act['percent_scoring_19_or_higher'] = 100 - df_mnps_act['percent_scoring_below_19']

# df_mnps_average_act_score = df_mnps_act[['file_year','subgroup','school_type','average_composite_score','percent_scoring_19_or_higher']].groupby(by=['file_year','subgroup','school_type']).mean('average_composite_score','percent_scoring_19_or_higher').reset_index()
# df_mnps_average_act_score

In [125]:
df_mnps_act_19_21[['all_public_composite','all_public','all_public_composite_avg']]

,all_public_composite,all_public,all_public_composite_avg
0,75437.2,3988.0,1892.0
1,80548.0,4265.0,1889.0
2,76730.5,4172.0,1839.0
3,72765.5,3955.0,1840.0
4,62789.5,3567.0,1760.0


In [135]:
df_state_act = df_merged_act_school.loc[df_merged_act_school['updated_district_number']==190].copy()
df_state_act['school_type'] = df_state_act['school_type'].replace('Public Virtual School','Public')

state_missing_act_tests = df_state_act.loc[df_state_act['percent_scoring_below_19']=='*'].groupby(by=['file_year','subgroup']).sum('valid_tests').reset_index().pivot(index='file_year',columns='subgroup',values='valid_tests')
state_all_act_tests = df_state_act.groupby(by=['file_year','subgroup']).sum('valid_tests').reset_index().pivot(index='file_year',columns='subgroup',values='valid_tests')
state_missing_pct_act = pd.concat([state_all_act_tests, state_missing_act_tests],axis=1)
state_missing_pct_act.columns = ['all','poc','econ_disadv','el_with_t1_t2','el','swd','all_missing','poc_missing','econ_disadv_missing','el_with_t1_t2_missing','el_missing','swd_missing']
state_missing_pct_act['acc_pct'] = ((state_missing_pct_act['all_missing'] / state_missing_pct_act['all']) * 100)

# df_state_act = df_state_act.loc[df_state_act['percent_scoring_below_19']!='*']
df_state_act = df_state_act.replace(['*'],[0])

# df_state_act['missing_19'] = pd.np.where(df_state_act.percent_scoring_below_19.str.contains('*'), 'missing')
for v in ['valid_tests','participation_rate','average_english_score','average_math_score','average_reading_score','average_science_score','average_composite_score','percent_scoring_21_or_higher','percent_scoring_below_19']:
    df_state_act[v] = pd.to_numeric(df_state_act[v],errors='coerce') 
    
df_state_act['cnt_scoring_21_or_higher']  = ((df_state_act['percent_scoring_21_or_higher'] / 100) * df_state_act['valid_tests']).round(0)
df_state_act['cnt_scoring_19_or_higher']  = ((1 - (df_state_act['percent_scoring_below_19'] / 100)) * df_state_act['valid_tests']).round(0)
df_state_act['composite_score_total']  = (df_state_act['average_composite_score']) * (df_state_act['valid_tests'])

df_state_act_19_21 = df_state_act[['school_number','school_name','subgroup','valid_tests','file_year','cnt_scoring_21_or_higher','cnt_scoring_19_or_higher','composite_score_total']]
df_state_act_19_21 = df_state_act_19_21.pivot_table(index=['file_year'],columns=['subgroup'],values=['cnt_scoring_21_or_higher','cnt_scoring_19_or_higher','composite_score_total','valid_tests'],aggfunc=np.sum).reset_index()
df_state_act_19_21.columns = ['file_year',
                             'all_state_19', 'poc_state_19', 'econ_disadv_state_19','el_t1_t2_state_19','el_state_19','swd_state_19',
                             
                             'all_state_21', 'poc_state_21', 'econ_disadv_state_21','el_t1_t2_state_21','el_state_21','swd_state_21',
                             
                             'all_state_composite', 'poc_state_composite', 'econ_disadv_state_composite','el_t1_t2_state_composite','el_state_composite','swd_state_composite',
                             
                             'all_public', 'poc_public', 'econ_disadv_public','el_t1_t2_public','el_public','swd_public']

for i in ['all', 'poc', 'econ_disadv','el_t1_t2','swd']: 
    df_state_act_19_21[i+'_state_19_pct'] = ((df_state_act_19_21[i+'_state_19']/df_state_act_19_21[i+'_public'])*100).round(0)
    df_state_act_19_21[i+'_state_21_pct'] = ((df_state_act_19_21[i+'_state_21']/df_state_act_19_21[i+'_public'])*100).round(0)  
    df_state_act_19_21[i+'_state_composite_avg'] = ((df_state_act_19_21[i+'_state_composite']/df_state_act_19_21[i+'_public'])).round(0)
  
df_state_act_19_21

,file_year,all_state_19,poc_state_19,econ_disadv_state_19,el_t1_t2_state_19,el_state_19,swd_state_19,all_state_21,poc_state_21,econ_disadv_state_21,el_t1_t2_state_21,el_state_21,swd_state_21,all_state_composite,poc_state_composite,econ_disadv_state_composite,el_t1_t2_state_composite,el_state_composite,swd_state_composite,all_public,poc_public,econ_disadv_public,el_t1_t2_public,el_public,swd_public,all_state_19_pct,all_state_21_pct,all_state_composite_avg,poc_state_19_pct,poc_state_21_pct,poc_state_composite_avg,econ_disadv_state_19_pct,econ_disadv_state_21_pct,econ_disadv_state_composite_avg,el_t1_t2_state_19_pct,el_t1_t2_state_21_pct,el_t1_t2_state_composite_avg,swd_state_19_pct,swd_state_21_pct,swd_state_composite_avg
0,2017,1351.0,668.0,486.0,83.0,NaN,74.0,1057.0,465.0,295.0,21.0,NaN,20.0,76197.7,45308.5,31679.8,6516.2,NaN,4883.1,4027.0,2592.0,1849.0,444.0,NaN,362.0,34.0,26.0,19.0,26.0,18.0,17.0,26.0,16.0,17.0,19.0,5.0,15.0,20.0,6.0,13.0
1,2018,1500.0,774.0,621.0,NaN,72.0,62.0,963.0,413.0,356.0,NaN,29.0,13.0,82653.4,49653.7,37637.6,NaN,7435.2,4468.2,4367.0,2851.0,2209.0,NaN,525.0,355.0,34.0,22.0,19.0,27.0,14.0,17.0,28.0,16.0,17.0,NaN,NaN,NaN,17.0,4.0,13.0
2,2019,1439.0,765.0,603.0,NaN,38.0,84.0,960.0,454.0,370.0,NaN,6.0,22.0,84547.5,53801.8,42928.7,NaN,6921.2,5781.4,4570.0,3120.0,2566.0,NaN,511.0,445.0,31.0,21.0,19.0,25.0,15.0,17.0,23.0,14.0,17.0,NaN,NaN,NaN,19.0,5.0,13.0
3,2020,1282.0,656.0,521.0,NaN,65.0,77.0,837.0,368.0,295.0,NaN,13.0,15.0,79769.0,49772.6,39094.1,NaN,8014.7,4715.8,4307.0,2913.0,2316.0,NaN,603.0,381.0,30.0,19.0,19.0,23.0,13.0,17.0,22.0,13.0,17.0,NaN,NaN,NaN,20.0,4.0,12.0
4,2021,1171.0,641.0,425.0,NaN,34.0,75.0,826.0,415.0,272.0,NaN,1.0,22.0,70651.9,42837.3,31884.5,NaN,6152.7,4810.8,4000.0,2646.0,2013.0,NaN,496.0,382.0,29.0,21.0,18.0,24.0,16.0,16.0,21.0,14.0,16.0,NaN,NaN,NaN,20.0,6.0,13.0


In [68]:
df_state_act_avg.head()

,school_name,subgroup,file_year,average_composite_score
0,Anderson County High School,All Students,2021,19.1
1,Anderson County High School,Black/Hispanic/Native American,2021,0.0
2,Anderson County High School,Economically Disadvantaged,2021,17.6
3,Anderson County High School,English Learners,2021,0.0
4,Anderson County High School,Students with Disabilities,2021,14.4


In [45]:
stats.ttest_ind(df_mnps_act.loc[(df_mnps_act['school_type']=='Public') & (df_mnps_act['subgroup']=='All Students') & (df_mnps_act['file_year']==2021)]['average_composite_score'],
                df_mnps_act.loc[(df_mnps_act['school_type']=='Public Charter') & (df_mnps_act['subgroup']=='All Students') & (df_mnps_act['file_year']==2021)]['average_composite_score'], trim=.2)

Ttest_indResult(statistic=-1.5806205489118936, pvalue=0.13137679671638547)

In [46]:
stats.ttest_ind(df_mnps_act.loc[(df_mnps_act['school_type']=='Public') & (df_mnps_act['subgroup']=='All Students') & (df_mnps_act['file_year']==2019)]['percent_scoring_19_or_higher'],
                df_mnps_act.loc[(df_mnps_act['school_type']=='Public') & (df_mnps_act['subgroup']=='All Students') & (df_mnps_act['file_year']==2021)]['percent_scoring_19_or_higher'], trim=.2)

Ttest_indResult(statistic=nan, pvalue=nan)

In [47]:
stats.ttest_ind(df_mnps_act.loc[(df_mnps_act['school_type']=='Public') & (df_mnps_act['subgroup']=='All Students') & (df_mnps_act['file_year']==2020)]['average_composite_score'],
                df_mnps_act.loc[(df_mnps_act['school_type']=='Public Charter') & (df_mnps_act['subgroup']=='All Students') & (df_mnps_act['file_year']==2020)]['average_composite_score'], trim=.2)

Ttest_indResult(statistic=-1.327886453280176, pvalue=0.20177052604148815)

In [48]:
stats.ttest_ind(df_mnps_act.loc[(df_mnps_act['school_type']=='Public') & (df_mnps_act['subgroup']=='All Students') & (df_mnps_act['file_year']==2019)]['average_composite_score'],
                df_mnps_act.loc[(df_mnps_act['school_type']=='Public Charter') & (df_mnps_act['subgroup']=='All Students') & (df_mnps_act['file_year']==2019)]['average_composite_score'], trim=.2)

Ttest_indResult(statistic=-1.3316994226800367, pvalue=0.2016206212911668)

In [49]:
stats.ttest_ind(df_mnps_act.loc[(df_mnps_act['school_type']=='Public') & (df_mnps_act['subgroup']=='All Students') & (df_mnps_act['file_year']==2018)]['average_composite_score'],
                df_mnps_act.loc[(df_mnps_act['school_type']=='Public Charter') & (df_mnps_act['subgroup']=='All Students') & (df_mnps_act['file_year']==2018)]['average_composite_score'], trim=.2)

Ttest_indResult(statistic=nan, pvalue=nan)

In [50]:
stats.ttest_ind(df_mnps_act.loc[(df_mnps_act['school_type']=='Public') & (df_mnps_act['subgroup']=='All Students') & (df_mnps_act['file_year']==2021)]['percent_scoring_below_19'],
                df_mnps_act.loc[(df_mnps_act['school_type']=='Public Charter') & (df_mnps_act['subgroup']=='All Students') & (df_mnps_act['file_year']==2021)]['percent_scoring_below_19'], trim=.2)

Ttest_indResult(statistic=nan, pvalue=nan)

In [51]:
stats.ttest_ind(df_mnps_act.loc[(df_mnps_act['school_type']=='Public Charter') & (df_mnps_act['subgroup']=='All Students') & (df_mnps_act['file_year']==2021)]['average_composite_score'],
                df_mnps_act.loc[(df_mnps_act['school_type']=='Public Charter') & (df_mnps_act['subgroup']=='All Students') & (df_mnps_act['file_year']==2020)]['average_composite_score'], trim=.2)

Ttest_indResult(statistic=-2.2356801952116525, pvalue=0.06674238871961843)

In [52]:
stats.ttest_ind(df_mnps_act.loc[(df_mnps_act['school_type']=='Public Charter') & (df_mnps_act['subgroup']=='All Students') & (df_mnps_act['file_year']==2020)]['average_composite_score'],
                df_mnps_act.loc[(df_mnps_act['school_type']=='Public Charter') & (df_mnps_act['subgroup']=='All Students') & (df_mnps_act['file_year']==2019)]['average_composite_score'], trim=.2)

Ttest_indResult(statistic=0.04022589933545843, pvalue=0.9698407415634583)

In [53]:
stats.ttest_ind(df_mnps_act.loc[(df_mnps_act['school_type']=='Public Charter') & (df_mnps_act['subgroup']=='All Students') & (df_mnps_act['file_year']==2021)]['average_composite_score'],
                df_mnps_act.loc[(df_mnps_act['school_type']=='Public Charter') & (df_mnps_act['subgroup']=='All Students') & (df_mnps_act['file_year']==2019)]['average_composite_score'], trim=.2)

## Charter School students performed on average -2.2 points lower at a 90% significance level 

Ttest_indResult(statistic=-2.200944982134069, pvalue=0.07001098848459264)

In [54]:
stats.ttest_ind(df_mnps_act.loc[(df_mnps_act['school_type']=='Public') & (df_mnps_act['subgroup']=='All Students') & (df_mnps_act['file_year']==2021)]['average_composite_score'],
                df_mnps_act.loc[(df_mnps_act['school_type']=='Public') & (df_mnps_act['subgroup']=='All Students') & (df_mnps_act['file_year']==2019)]['average_composite_score'], trim=.2)

## Public School students performed on average -1.7 points lower at a 90% significance level 

Ttest_indResult(statistic=-1.7198528341816568, pvalue=0.09649598450363786)

In [136]:
with pd.ExcelWriter('../data_for_analysis/act.xlsx') as writer:  
    df_mnps_act.to_excel(writer, sheet_name='mnps_act')
    df_mnps_act_19_21.to_excel(writer, sheet_name='mnps_19_21')
    df_state_act_19_21.to_excel(writer, sheet_name='state')